In [2]:
# pip install keras_tuner
# pip install pandas
# pip install numpy
# pip install tensorflow


In [ ]:
def handle_input_window_size(train_input, window_size):
        """ make window sized data from 2d data

        """
        print(window_size)
        if window_size > 1:
            condition = len(train_input.shape) > 1
            if not condition:
                train_input=train_input.reshape(-1, 1)
            m, n = train_input.shape
            s0, s1 = train_input.strides
            _shape = (m - window_size + 1, window_size, n)
            _strides = (s0, s0, s1)
            return np.lib.stride_tricks.as_strided(train_input, shape=_shape, strides=_strides)
        else:
            train_input = np.expand_dims(train_input, 1)
        return train_input

In [ ]:
import pandas as pd
import keras_tuner as kt
import tensorflow as tf
import numpy as np
# import tensorflow_model_optimization as tfmot
data=pd.read_csv('pjm-5min.csv')
data.drop(data.iloc[:,-4:-1],axis=1,inplace=True)
data.dropna(inplace=True)
window_size=6


class MyTuner(kt.Hyperband):
  def run_trial(self, trial, *args, **kwargs):
    # You can add additional HyperParameters for preprocessing and custom training loops
    # via overriding `run_trial`
    kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 5, 96, step=30)
    # kwargs['epochs'] = trial.hyperparameters.Int('epochs', 10, 60,step=10)
    super(MyTuner, self).run_trial(trial, *args, **kwargs)

    
def build_model(hp):
    # window_size= hp.Choice("window_size",[3,6,12,24])
    """Builds a convolutional model."""
    inputs1 = tf.keras.Input(shape=(data.shape[1]-2))
    inputs2=  tf.keras.Input(shape=(window_size,1))

    x1 = inputs1
    x1 = tf.keras.layers.Dense(
          units=hp.Int("units_den1" , 2, 100, step=5),
          activation=hp.Choice("active_func_den1",["relu","tanh","sigmoid"]),
          )(x1)

    x2=inputs2
    x2 = tf.keras.layers.LSTM(
          units=hp.Int("units_LSTM" , 5, 95, step=5)
        , activation=hp.Choice("lstm_active_func",["relu","tanh","sigmoid"])
        , recurrent_activation=hp.Choice("lstm_rec_active_func",["tanh","sigmoid"])
        , dropout=0.1
        # ,stateful=hp.Choice("lstm_stateful",[True,False])
        ,return_sequences=True
    )(x2)
    x2=tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Dense(
          units=hp.Int("units_den2" , 2, 100, step=5),
          activation=hp.Choice("active_func_den2",["relu","tanh","sigmoid"]),
          )(x2)

    combinedInput = tf.keras.layers.concatenate([x1, x2])
    x = tf.keras.layers.Dense(
          units=hp.Int("units_den3" , 2, 100, step=5),
          activation=hp.Choice("active_func_den3",["relu","tanh","sigmoid"]),
          )(combinedInput)



    outputs = tf.keras.layers.Dense(1, activation="linear")(x)

    model = tf.keras.Model([inputs1,inputs2], outputs)
    # model= tfmot.quantization.keras.quantize_model(model)



    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        hp.Choice("initial_learning_rate", values=[1e-2,1e-3,5e-3]),
        hp.Int("decay_steps", 10 , 100, step=20 ),
        hp.Float("decay_rate" , 0.7 ,1, step=0.05 )
       )
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    # optimizer = hp.Choice("optimizer", ['adam', 'sgd'])
    # if hp.Choice("optimizer", ['adam', 'sgd']) == "adam":
    #     optimizer = tf.keras.optimizers.Adam(
    #         hp.Choice("learning_rate", values=[1e-3,5e-4])
    #     )
    # else:
    #     optimizer = tf.keras.optimizers.SGD(
    #         hp.Choice("learning_rate", values=[1e-3,5e-4])
    #     )

    classes_range=[-float('inf'),hp.Int("range", 40 , 200, step=10 ),float('inf')]
    classes_factor=[1,hp.Int("factor", 2 , 15, step=1 )]
    def custom_loss(y_true, y_pred):
        y_true = tf.cast(y_true, y_pred.dtype)
        loss = tf.square(y_true - y_pred)
        coef = tf.zeros_like(loss)
        for i in range(1, len(classes_range)):
            index = tf.where(tf.math.logical_and(
                tf.greater_equal(y_true, tf.constant(classes_range[i - 1], dtype = y_pred.dtype)),
                tf.less_equal(y_true, tf.constant(classes_range[i], dtype = y_pred.dtype))))
            coef = tf.tensor_scatter_nd_update(coef, index,
                                               tf.ones(tf.shape(index)[0], dtype = 'float32') * classes_factor[i - 1])

        loss = tf.multiply(coef, loss)
        return tf.reduce_mean(loss)

    model.compile(
        optimizer, loss=custom_loss, metrics=["mape"]
    )
    return model


tuner = MyTuner(
    hypermodel=build_model,
    objective="mape",
    max_epochs=100,
    factor=3,
    hyperband_iterations=1,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory="results_dir",
    project_name="qr",
    overwrite=True
)

# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train_nonprice=data.iloc[window_size-1:-288].drop(data.columns[[0,-1]],axis=1)
x_train_price=data.iloc[:-288,0]*10
y_train=data.iloc[window_size-1:-288]['Target']

x_test_nonprice=data.iloc[-288+window_size-1:].drop(data.columns[[0,-1]],axis=1)
x_test_price=data.iloc[-288:,0]*10
y_test=data.iloc[-288+window_size-1:]['Target']

x_train_price=handle_input_window_size(x_train_price.values, window_size)
x_test_price=handle_input_window_size(x_test_price.values, window_size)

tuner.search(
    [x_train_nonprice,x_train_price],
    y_train,
    validation_data=([x_test_nonprice,x_test_price], y_test),
    # callbacks=[tf.keras.callbacks.EarlyStopping("val_mape")],
)